In [1]:
from dataset import read_annotations_file, tokenize_sentences
import numpy as np
from model import ImageCaptioningSystem
import pytorch_lightning as pl
from pathlib import Path
from dataset import NWPU_Captions
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
from evaluation import eval_validation
import matplotlib.pyplot as plt
import torch
import pandas as pd
import seaborn as sns
import tqdm

/home/fe/hufe/miniconda3/envs/remote-sensing/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package wordnet to /home/fe/hufe/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/fe/hufe/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
%cd ..

/home/fe/hufe/Documents/semester11/activelearning_ic


In [5]:
annotations = read_annotations_file("NWPU-Captions/NWPU_images/", "NWPU-Captions/dataset_nwpu.json")
tk = annotations['test']['sentences']
ids = np.array(annotations['test']['img_id'])
order = ids.argsort()
ordered_sentences = np.array(tk)[order]

In [6]:
for i in range(5):
    with open(f'evaluation/test_references_{i}.txt', 'w') as f:
        for x in ordered_sentences[:,i]:
            x = x.encode('ascii', 'ignore').decode('ascii')
            f.write(x)
            f.write('\n')

In [7]:
data_path = "NWPU-Captions/"
images_path = Path(data_path, "NWPU_images")
annotations_path = Path(data_path, "dataset_nwpu.json")

test_set = NWPU_Captions(
    root=images_path,
    annotations_file=annotations_path,
    split="test",
    transform=ToTensor(),
)

test_loader = DataLoader(
    test_set, batch_size=12, shuffle=False, num_workers=4
)

/home/fe/hufe/miniconda3/envs/remote-sensing/lib/python3.10/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [9]:
# trainer = pl.Trainer(accelerator="cuda",)
model = ImageCaptioningSystem.load_from_checkpoint("test/full.ckpt", lr=0.001, device_type='cpu', sampling_method='cluster')
# results = trainer.predict(model, test_loader)

Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v1.9.2. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file test/full.ckpt`


In [ ]:
with tqdm.tqdm(total=len(test_loader)) as pbar:
    for batch in test_loader:
        pixel_values, sentences_token, img_ids, sentences_ids = batch
        print(img_ids)
        pixel_values = pixel_values.squeeze(dim=1)
        bs = pixel_values.shape[0]
        label = sentences_token[:, 0, :].long().contiguous()

        with torch.no_grad():
            out = model.model(
                    pixel_values=pixel_values, labels=label, output_hidden_states=True
                )

        logits = out.logits
        logits_softmax = torch.nn.functional.softmax(logits, dim=2)
        pbar.update(1)


In [60]:
bleu, meteor, rogue = eval_validation([f'test/test_references_{i}.txt' for i in range(5)], 'test')
soa_bleu, soa_meteor, soa_rouge = 0.463, 0.337, 0.628

c:\Users\Lorenz\miniconda3\envs\project_remote_sensing\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\Users\Lorenz\miniconda3\envs\project_remote_sensing\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\Users\Lorenz\miniconda3\envs\project_remote_sensing\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lowe

In [63]:
pd.DataFrame({'bleu':[bleu, soa_bleu], 'meteor':[meteor, soa_meteor], 'rogue':[rogue, soa_rouge], 'model':['ours', 'soa']})

,bleu,meteor,rogue,model
0,0.774181,0.453994,0.671007,ours
1,0.463000,0.337000,0.628000,soa
